In [1]:
import sys
import pymol
_stdouterr = sys.stdout, sys.stderr
pymol.finish_launching(['/usr/bin/pymol', '-q'])
sys.stdout, sys.stderr = _stdouterr

# load something into the PyMOL window
from pymol import cmd


Gtk-Message: 12:59:52.714: Failed to load module "pk-gtk-module"
Cannot open file '/home/vit/Projects/cryptic-nn/src/prediction-analysis/data/pymol/icons/icon2.svg', because: No such file or directory
Cannot open file '/home/vit/Projects/cryptic-nn/src/prediction-analysis/data/pymol/icons/icon2.svg', because: No such file or directory


Could not read PyMOL stylesheet.
DEBUG: PYMOL_DATA='./data'


## Clustering
Cluster the predictions into pockets.

In [2]:
import numpy as np
from sklearn.cluster import DBSCAN, AgglomerativeClustering

EPSILON = 7  # Max distance for neighbors (adjust as needed)
MIN_SAMPLES = 3  # Minimum points to form a cluster         (adjust as needed)

def compute_clusters(points: list[list[float]], prediction_scores: list[float]):
    # This function computes clusters for the given points and prediction scores
    points_array = np.array(points)
    scores_array = np.array(prediction_scores).reshape(-1, 1)
    stacked = np.hstack((points_array, scores_array))  # Combine coordinates with scores

    high_score_mask = stacked[:, 3] > 0.65  # TODO: tweak this
    high_score_points = stacked[high_score_mask][:, :3]  # Extract only (x, y, z) coordinates

    dbscan = DBSCAN(eps=EPSILON, min_samples=MIN_SAMPLES)
    # dbscan = AgglomerativeClustering(distance_threshold=EPSILON, n_clusters=None, linkage='single')
    labels = dbscan.fit_predict(high_score_points)

    # Initialize all labels to -1
    all_labels = -1 * np.ones(len(points), dtype=int)
    # Assign cluster labels to high score points
    all_labels[high_score_mask] = labels
    labels = all_labels

    return labels

 Detected OpenGL version 4.6. Shaders available.
 Geometry shaders not available
 Detected GLSL version 4.60.


PyMOL>set fetch_path, /home/vit/Projects/ahoj2-extraction/src/analysis/cif_files
 Setting: fetch_path set to /home/vit/Projects/ahoj2-extraction/src/analysis/cif_files.


In [ ]:
import json
import sys, os
import numpy as np

from biotite.structure import get_residues, get_residue_starts
from biotite.structure.io.pdbx import get_structure
import biotite.structure.io.pdbx as pdbx
import biotite.database.rcsb as rcsb
import biotite


DATASET = 'cryptobench'
CIF_FILES = '/home/vit/Projects/deeplife-project/data/cif_files'
PREDICTIONS_PATH = '/home/vit/Projects/cryptic-nn/data/predictions/ESM2-3B-extended-finetuning'
COLORS = ['red', 'pink', 'purple', 'purpleblue', 'raspberry', 'ruby', 'salmon', 'sand', 'skyblue', 'slate', 'smudge', 'splitpea', 'sulfur', 'teal', 'tv_blue', 'tv_green', 'tv_orange', 'tv_red', 'tv_yellow']
with open(f'../../datasets/{DATASET}-dataset/folds/test.json', 'r') as json_file:
    dataset = json.load(json_file)

skip = False
# skip = True

for apo_structure, holo_structures in dataset.items():

    # finished analysis at: '5wbmB' structure
    if skip:
        if apo_structure == '5wm9':
            skip = False
        else:
            continue
    chain_id = holo_structures[0]['apo_chain']

    # skip multichain structures
    if '-' in chain_id:
        continue


    cif_file_path = rcsb.fetch(apo_structure, "cif", target_path=CIF_FILES)

    cif_file = pdbx.CIFFile.read(cif_file_path)

    auth = get_structure(cif_file, model=1)
    auth = auth[
            (auth.chain_id == chain_id) &
            (biotite.structure.filter_amino_acids(auth))]
    
    protein_id = f'{apo_structure}{chain_id}'
    # skip if no residues left
    if len(auth) == 0: 
        print(f'No residues left for {protein_id}')
        continue

    # filter to get correct chain; filter only for peptides
    auth_residues_only = auth[get_residue_starts(auth)]

    predictions = np.load(f'{PREDICTIONS_PATH}/predictions/{protein_id}.npy') > 0.5
    
    assert len(predictions) == len(auth_residues_only), f"Length of auth residues and predictions do not match for {protein_id}: {len(auth_residues_only)} vs {len(predictions)}"
    predicted_binding_residues = auth_residues_only[predictions]
    predicted_binding_residue_coords, predicted_binding_residue_auth_labels = predicted_binding_residues.coord, predicted_binding_residues.res_id

    clusters = compute_clusters(predicted_binding_residue_coords, predictions[predictions])
    
    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.fetch(protein_id)
    cmd.zoom(protein_id)
    cmd.color('grey', protein_id)

    for i in range(-1, max(clusters) + 1):
        cluster_residue_auth_labels = predicted_binding_residue_auth_labels[clusters == i]
        if i == -1 and len(cluster_residue_auth_labels) == 0:
            continue
        cmd.color(COLORS[i + 1], f'{protein_id} and resi {"+".join([str(i) for i in cluster_residue_auth_labels])}')
        
    cmd.show('surface', protein_id)
    input(">Press Enter for the next protein...\n")


 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF


## Sphere around the pocket's centre
Draw a sphere around the pocket's centre and include all residues inside the sphere into the pocket. 

In [ ]:
import json
import sys, os
import numpy as np

from biotite.structure import get_residues, get_residue_starts
from biotite.structure.io.pdbx import get_structure
import biotite.structure.io.pdbx as pdbx
import biotite.database.rcsb as rcsb
import biotite


SPHERE_RADIUS_RATIO = 0.75
DATASET = 'cryptobench'
CIF_FILES = '/home/vit/Projects/deeplife-project/data/cif_files'
PREDICTIONS_PATH = '/home/vit/Projects/cryptic-nn/data/predictions/ESM2-3B-extended-finetuning'
COLORS = ['red', 'pink', 'purple', 'purpleblue', 'raspberry', 'ruby', 'salmon', 'sand', 'skyblue', 'slate', 'smudge', 'splitpea', 'sulfur', 'teal', 'tv_blue', 'tv_green', 'tv_orange', 'tv_red', 'tv_yellow']
with open(f'../../datasets/{DATASET}-dataset/folds/test.json', 'r') as json_file:
    dataset = json.load(json_file)

def spread_pocket(pocket_residues, other_residues):
    centroid = np.mean(pocket_residues, axis=0)

    # get radius of the sphere
    radius = np.max(np.linalg.norm(pocket_residues - centroid, axis=1)) * SPHERE_RADIUS_RATIO

    additional_residues_mask = np.linalg.norm(centroid - other_residues.coord, axis=1) < radius
    additional_residue_auth_labels = other_residues[additional_residues_mask].res_id

    return list(set(additional_residue_auth_labels).union(set(pocket_residues.res_id)))


skip = False
# skip = True

for apo_structure, holo_structures in dataset.items():

    # finished analysis at: '5wbmB' structure
    if skip:
        if apo_structure == '5wm9':
            skip = False
        else:
            continue
    chain_id = holo_structures[0]['apo_chain']

    # skip multichain structures
    if '-' in chain_id:
        continue


    cif_file_path = rcsb.fetch(apo_structure, "cif", target_path=CIF_FILES)

    cif_file = pdbx.CIFFile.read(cif_file_path)

    auth = get_structure(cif_file, model=1)
    auth = auth[
            (auth.chain_id == chain_id) &
            (biotite.structure.filter_amino_acids(auth))]
    
    protein_id = f'{apo_structure}{chain_id}'
    # skip if no residues left
    if len(auth) == 0: 
        print(f'No residues left for {protein_id}')
        continue

    # filter to get correct chain; filter only for peptides
    auth_residues_only = auth[get_residue_starts(auth)]

    predictions = np.load(f'{PREDICTIONS_PATH}/predictions/{protein_id}.npy') > 0.5
    
    assert len(predictions) == len(auth_residues_only), f"Length of auth residues and predictions do not match for {protein_id}: {len(auth_residues_only)} vs {len(predictions)}"
    predicted_binding_residues = auth_residues_only[predictions]
    predicted_binding_residue_coords, predicted_binding_residue_auth_labels = predicted_binding_residues.coord, predicted_binding_residues.res_id

    # -1 denotes residues that are not part of any cluster
    clusters = compute_clusters(predicted_binding_residue_coords, predictions[predictions])
    
    cluster_index = 0

    # compute centroid
    first_cluster = predicted_binding_residue_coords[clusters == cluster_index]
    centroid = np.mean(first_cluster, axis=0)

    # get radius of the sphere
    radius = np.max(np.linalg.norm(first_cluster - centroid, axis=1)) * SPHERE_RADIUS_RATIO

    additional_residues_mask = np.linalg.norm(centroid - auth_residues_only.coord, axis=1) < radius

    additional_residue_auth_labels = auth_residues_only[additional_residues_mask].res_id
    # print(f'Additional residues: {additional_residue_auth_labels}, {predicted_binding_residue_auth_labels[clusters == cluster_index]}')
    # print(f'filtered additional residues = ', [i for i in additional_residue_auth_labels if i not in predicted_binding_residue_auth_labels[clusters == cluster_index]])
    
    cmd.reinitialize()
    cmd.set('fetch_path', cmd.exp_path(CIF_FILES), quiet=0)
    cmd.fetch(protein_id)
    cmd.zoom(protein_id)
    cmd.color('grey', protein_id)

    for cluster_index in range(-1, max(clusters) + 1):
        if cluster_index == -1 and len(predicted_binding_residue_auth_labels[clusters == cluster_index]) == 0:
            continue

        # TODO: UNCOMENT THIS LINE
        if cluster_index > 0: break

        if cluster_index != -1:
            # compute centroid
            cluster = predicted_binding_residue_coords[clusters == cluster_index]
            centroid = np.mean(cluster, axis=0)

            # get radius of the sphere
            radius = np.max(np.linalg.norm(cluster - centroid, axis=1)) * SPHERE_RADIUS_RATIO
            additional_residues_mask = np.linalg.norm(centroid - auth_residues_only.coord, axis=1) < radius
            # additional residue auth labels
            additional_residue_auth_labels = auth_residues_only[additional_residues_mask].res_id
            # original cluster auth labels
            cluster_residue_auth_labels = predicted_binding_residue_auth_labels[clusters == cluster_index]

            # merge those two
            # TODO: UNCOMENT THIS LINE
            # cluster_residue_auth_labels = list(set(cluster_residue_auth_labels).union(set(additional_residue_auth_labels)))
            # TODO: DELETE
            cmd.color('red', f'{protein_id} and resi {"+".join([str(i) for i in additional_residue_auth_labels])}')
            cmd.color('green', f'{protein_id} and resi {"+".join([str(i) for i in cluster_residue_auth_labels])}')

        else:
            # for -1 cluster, we need to use the original auth labels
            cluster_residue_auth_labels = predicted_binding_residue_auth_labels[clusters == cluster_index]

            # TODO: DELETE
            cmd.color('grey', f'{protein_id} and resi {"+".join([str(i) for i in cluster_residue_auth_labels])}')

        # TODO : UNCOMENT THIS LINE
        # cmd.color(COLORS[i + 1], f'{protein_id} and resi {"+".join([str(i) for i in cluster_residue_auth_labels])}')

    cmd.show('surface', protein_id)
    input(">Press Enter for the next protein...\n")


 Setting: fetch_path set to /home/vit/Projects/deeplife-project/data/cif_files.
 ExecutiveLoad-Detail: Detected mmCIF
7qoqA and resi 45+46+47+48+49+50+51+54+55+76+78+110+111+112+113+114+115+116+129+130+131+132+133+134+135+136+137+160+161+162+178+180+181+202+203+204+205+206+207+208+222+223+224+225+226+227+253+254+255+260+263+264


## Sphere around each residue
Draw a small sphere around each residue. If the sphere contains more than `N` binding residues, add the residue to the pocket.